# Creazione del dataset unito dai dataset csv

In [3]:
import os
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import ast
import torch
import pickle

In [ ]:
# Percorsi ai file CSV
recipes_path = 'Datasets\RAW_recipes.csv'
interactions_path = 'Datasets\RAW_interactions.csv'

# Caricamento dei dataset
recipes_df = pd.read_csv(recipes_path)
interactions_df = pd.read_csv(interactions_path)

# Visualizza le prime righe di ciascun dataset per verificare il contenuto
print(recipes_df.head())
print(interactions_df.head())

In [ ]:
# Nomi delle colonne nei due dataset
print(recipes_df.columns)
print(interactions_df.columns)

# Verifica delle dimensioni dei dataset
print(f"Recipes dataset shape: {recipes_df.shape}")
print(f"Interactions dataset shape: {interactions_df.shape}")

In [ ]:
# Unione dei due dataset usando 'id' dal dataset delle ricette e 'recipe_id' dal dataset delle interazioni
merged_df = pd.merge(recipes_df, interactions_df, left_on='id', right_on='recipe_id')

# Visualizza le prime righe del dataset unito
print(merged_df.shape)

In [ ]:
# Controlla se ci sono valori nulli nel dataset unito
print(merged_df.isnull().sum())

# Analizza la distribuzione dei rating
print(merged_df['rating'].describe())

# Mostra qualche esempio di ricette con rating
print(merged_df[['name', 'ingredients', 'rating']].head())

In [ ]:
# Rimozione di righe duplicate
merged_df = merged_df.drop_duplicates()

# Verifica del numero di righe dopo la rimozione
print(merged_df.shape)

In [ ]:
# Controlla nuovamente per valori nulli
print(merged_df.isnull().sum())

# Rimuovi righe con valori nulli nelle colonne cruciali (ad esempio, ingredienti, passaggi e valutazioni)
merged_df = merged_df.dropna(subset=['ingredients', 'steps', 'rating'])

# Verifica di nuovo la presenza di nulli
print(merged_df.isnull().sum())

In [ ]:
# Salva il dataset unito
merged_df.to_csv('Datasets\working_dataset.csv', index=False)

print("Dataset unito e salvato con successo.")

# Import del working Dataset e tokenization degli ingredienti e steps

In [ ]:
import os
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import ast
import torch
import pickle

In [5]:
# Carica il dataset unito
merged_df = pd.read_csv('Datasets\working_dataset.csv')

In [6]:
# Converti le stringhe in liste di stringhe
merged_df['ingredients'] = merged_df['ingredients'].apply(ast.literal_eval)
merged_df['steps'] = merged_df['steps'].apply(ast.literal_eval)

In [ ]:
def format_recipe(ingredients, steps):
    # Concatena gli ingredienti in una stringa separata da virgole
    ingredients_str = ', '.join(ingredients)

    # Concatena gli steps in una stringa separata da numeri
    steps_str = '. '.join([f"{i+1}. {step}" for i, step in enumerate(steps)])

    # Combina tutto in una singola stringa
    return f"Ingredients: {ingredients_str}. Steps: {steps_str}."

# Applica la funzione a tutte le righe del dataframe
merged_df['formatted'] = merged_df.apply(lambda row: format_recipe(row['ingredients'], row['steps']), axis=1)

# Converti in una lista di stringhe pronte per il training
formatted_recipes = merged_df['formatted'].tolist()

print(formatted_recipes[0])

In [ ]:
# Carica il tokenizer GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# aggiungo un padding token al tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenizza le ricette formattate
encodings = tokenizer(formatted_recipes, truncation=True, padding=True, max_length=512, return_tensors='pt')

# Salva encodings su un file
torch.save(encodings, 'encodings.pt')

tokenizer.save_pretrained('results/tokenizer/')

# import del tokenizer e allenamento del modello gpt2

In [13]:
import os
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import ast
import torch
import pickle

In [ ]:
# Carico il modello GPT2
model = GPT2LMHeadModel.from_pretrained('gpt2')
# Carico il tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('results/tokenizer/')
# Carico encodings da file
encodings = torch.load('encodings.pt')

#classe utilizzata per allenare il modello di machine learning
class CustomTextDataset(torch.utils.data.Dataset):
    #inizializzo l'istanza della classe con gli encodings
    def __init__(self, encodings):
        self.encodings = encodings

    #Recupera un elemento dal dataset dato un indice idx
    def __getitem__(self, idx):
        return {key: val[idx].clone().detach() for key, val in self.encodings.items()}

    #Restituisce la lunghezza del dataset
    def __len__(self):
        return len(self.encodings.input_ids)

dataset = CustomTextDataset(encodings)

# Configura l'allenamento
training_args = TrainingArguments(
    output_dir='results/training/',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=10000,
    save_total_limit=2,
)

# Crea un nuovo trainer e carica lo stato precedente
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

In [ ]:
# Controlla se esiste un checkpoint altrimenti inizia da zero
checkpoint_dir = 'results/training/'
checkpoints = [os.path.join(checkpoint_dir, d) for d in os.listdir(checkpoint_dir) if d.startswith("checkpoint-")]
checkpoints = sorted(checkpoints, key=lambda x: int(x.split('-')[-1]))

if checkpoints:
    last_checkpoint = checkpoints[-1]
    print(f"Riprendo l'addestramento dall'ultimo checkpoint: {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("Nessun checkpoint trovato, inizio un nuovo addestramento")
    trainer.train()

In [ ]:
#salvo gli stati finali su file per poter usarli in seguito
model.save_pretrained('ModelGPT/model_after_train')
tokenizer.save_pretrained('ModelGPT/tokenizer_after_train')
trainer.save_state()

# Creazione della correlazion matrix per vedere la correlazione tra ingredienti/steps/ratings

In [ ]:
import os
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import ast
import torch
import pickle

In [2]:
# Carica dataset
merged_df = pd.read_csv('Datasets\working_dataset.csv')

In [3]:
# Funzione per pulire e trasformare la stringa in una lista
def clean_ingredient_steps_string(ingredient_str):
    # Rimuovere le virgolette in eccesso e trasformare la stringa in lista
    try:
        # Prima rimuovi eventuali spazi e virgolette non necessarie
        ingredient_str = ingredient_str.strip("[]").replace("'", "").strip()

        # Poi, separa gli ingredienti o gli steps se sono stati convertiti in una singola stringa
        ingredients = [ing.strip() for ing in ingredient_str.split(',')]

        return ingredients
    except Exception as e:
        print(f"Errore nella pulizia dell'ingrediente: {e}")
        return []

# Applicare la funzione di pulizia su tutta la colonna 'ingredients_list' e 'steps'
merged_df['ingredients_list'] = merged_df['ingredients'].apply(clean_ingredient_steps_string)
merged_df['steps_list'] = merged_df['steps'].apply(clean_ingredient_steps_string)

In [ ]:
# Verifica del risultato
print(type(merged_df['ingredients_list'][0]))  # Controllo il tipo della colonna ingredient_list
print(type(merged_df['steps_list'][0]))  # Controllo il tipo della colonna steps_list

# Verifica del risultato
print(merged_df['ingredients_list'][0])  # Ora dovrebbe essere una lista di stringhe
print(merged_df['steps_list'][0])  # Ora dovrebbe essere una lista di stringhe

In [5]:
# 1. Preprocessing del Testo

# Tokenizzazione e padding per ingredients_list
ingredient_tokenizer = Tokenizer()
ingredient_tokenizer.fit_on_texts(merged_df['ingredients_list'])
ingredient_sequences = ingredient_tokenizer.texts_to_sequences(merged_df['ingredients_list'])
ingredient_maxlen = max(len(seq) for seq in ingredient_sequences)
ingredient_padded = pad_sequences(ingredient_sequences, maxlen=ingredient_maxlen)

In [6]:
# Tokenizzazione e padding per steps_list
steps_tokenizer = Tokenizer()
steps_tokenizer.fit_on_texts(merged_df['steps_list'])
steps_sequences = steps_tokenizer.texts_to_sequences(merged_df['steps_list'])
steps_maxlen = max(len(seq) for seq in steps_sequences)
steps_padded = pad_sequences(steps_sequences, maxlen=steps_maxlen)

In [ ]:
#stampo il valore perchè devo prendere questo valore e salvarlo dopo
print(f"Lunghezza massima per gli ingredienti: {ingredient_maxlen}")
print(f"Lunghezza massima per i passi: {steps_maxlen}")

In [ ]:
# Salva il tokenizer per gli ingredienti su file
with open('ModelClassifier/tokenizer_ingredients.pkl', 'wb') as file:
    pickle.dump(ingredient_tokenizer, file)

# Salva il tokenizer per gli steps su file
with open('ModelClassifier/tokenizer_steps.pkl', 'wb') as file:
    pickle.dump(steps_tokenizer, file)

In [ ]:
# Mappa per ingredienti e passi
def get_feature_names(tokenizer, maxlen):
    index_word = {index: word for word, index in tokenizer.word_index.items()}
    return [index_word.get(i, f'unknown_{i}') for i in range(1, maxlen + 1)]

ingredient_names = get_feature_names(ingredient_tokenizer, ingredient_maxlen)
steps_names = get_feature_names(steps_tokenizer, steps_maxlen)

# Creazione di DataFrame
ingredients_df = pd.DataFrame(ingredient_padded, columns=ingredient_names)
steps_df = pd.DataFrame(steps_padded, columns=steps_names)

# Combinazione delle caratteristiche e dei rating
features_df = pd.concat([ingredients_df, steps_df], axis=1)
features_df['rating'] = merged_df['rating']

# Calcolo e visualizzazione delle correlazioni
correlation_matrix = features_df.corr()
rating_correlation = correlation_matrix['rating'].drop('rating')
sorted_correlation = rating_correlation.sort_values(ascending=False)

print("Correlazione tra ingredienti, passi e valutazioni:")
print(sorted_correlation)

In [12]:
# Salva la rating_correlation in un file
rating_correlation.to_pickle('ModelClassifier/rating_correlation.pkl')

# Import GPT2 model e il classificatore per generare ricette per poi trovare la migliore

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import ast
import torch
import pickle

In [3]:
# Carica il modello e il tokenizer da file
gpt2Model = GPT2LMHeadModel.from_pretrained('ModelGPT/model_after_train')
tokenizer = GPT2Tokenizer.from_pretrained('ModelGPT/tokenizer_after_train')

# Carica il classificatore da file
loaded_correlation = pd.read_pickle('ModelClassifier/rating_correlation.pkl')

# Carica il tokenizer per gli ingredienti da file
with open('ModelClassifier/tokenizer_ingredients.pkl', 'rb') as file:
    tokenizer_ingredients = pickle.load(file)
    
# Carica il tokenizer per i passaggi da file
with open('ModelClassifier/tokenizer_steps.pkl', 'rb') as file:
    tokenizer_steps = pickle.load(file)

In [ ]:
#max_new_tokens: numero massimo di nuovi token da generare
#temperature: Controlla la casualità delle previsioni. Un valore più basso rende il testo più deterministico, mentre un valore più alto lo rende più vario.
#top_k: Limita le scelte del modello ai k migliori risultati (per migliorare la qualità delle generazioni).
#top_p: Percentuale cumulativa di probabilità considerata per la scelta dei token. un valore alto aumenta la varietà e creatività includendo però i token meno probabili e viceversa


def generate_recipe(prompt, max_new_tokens=100, temperature=0.8, top_k=50, top_p=0.9):
    input_text = f"Ingredients: {prompt}. Steps:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    sample_output = gpt2Model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        do_sample=True,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.ones_like(input_ids)
    )

    generated_text = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    generated_text = " ".join(generated_text.split()).strip()  # Rimuovi spazi extra

    return generated_text

# Genera 5 ricette chiamando la funzione 5 volte
prompt = "tomato, potato, milk, salt"
generated_recipes = [generate_recipe(prompt) for _ in range(5)]

# Stampa le ricette generate
for i, recipe in enumerate(generated_recipes, 1):
    print(f"Generated Recipe {i}:\n{recipe}\n")

In [ ]:
import re

#funzione per estrarre dalle ricette generate gli ingredienti e gli steps
def extract_ingredients_and_steps_from_recipes(recipes):
    all_ingredients_lists = []
    all_steps_lists = []

    for text in recipes:
        # Separare la sezione degli ingredienti e degli steps
        ingredients_section = re.search(r'Ingredients: (.+?)\. Steps:', text)
        steps_section = re.search(r'Steps: (.+)', text)

        if ingredients_section:
            ingredients_text = ingredients_section.group(1).strip()
            # Convertire gli ingredienti in una lista di stringhe
            ingredients_list = [ingredient.strip() for ingredient in ingredients_text.split(',') if ingredient.strip()]
            all_ingredients_lists.append(ingredients_list)
        else:
            all_ingredients_lists.append([])

        if steps_section:
            steps_text = steps_section.group(1).strip()
            # Separare gli steps e rimuovere numeri e punti
            steps_raw = re.split(r'\d+\.\s*', steps_text)  # Usa regex per separare usando numeri e punto
            # Filtrare e pulire gli steps
            steps_list = [step.strip() for step in steps_raw if step.strip()]
            all_steps_lists.append(steps_list)
        else:
            all_steps_lists.append([])

    return all_ingredients_lists, all_steps_lists

all_ingredients_lists, all_steps_lists = extract_ingredients_and_steps_from_recipes(generated_recipes)

print("All Ingredients Lists:", all_ingredients_lists)
print("All Steps Lists:", all_steps_lists)

In [ ]:
print(all_steps_lists[0])
print(all_steps_lists[1])
print(all_steps_lists[2])
print(all_steps_lists[3])
print(all_steps_lists[4])

In [ ]:
#rimuove i punti finali negli steps per poterli tokenizzare e usare dopo
def remove_trailing_periods_from_steps(nested_steps_list):
    if not all(isinstance(sublist, list) for sublist in nested_steps_list) or \
       not all(isinstance(step, str) for sublist in nested_steps_list for step in sublist):
        raise ValueError("La lista deve essere una lista di liste di stringhe.")

    # Applica la pulizia a ciascuna sotto-lista
    cleaned_nested_list = [
        [step.strip().rstrip('.') for step in sublist]
        for sublist in nested_steps_list
    ]

    return cleaned_nested_list
# Rimuovi i punti finali
cleaned_all_steps_lists = remove_trailing_periods_from_steps(all_steps_lists)

# Stampa il risultato
for i, steps in enumerate(cleaned_all_steps_lists, 1):
    print(f"Recipe {i} Steps: {steps}")

In [74]:
# Tokenizzazione e padding
new_ingredients_seq = tokenizer_ingredients.texts_to_sequences(all_ingredients_lists)
new_steps_seq = tokenizer_steps.texts_to_sequences(cleaned_all_steps_lists)

#il Calcolo della lunghezza massima per gli ingredienti e i passi è già stata fatta prima nel notebook
ingredient_maxlen = 43
steps_maxlen = 218

# Padding
new_ingredients_padded = pad_sequences(new_ingredients_seq, maxlen=ingredient_maxlen)
new_steps_padded = pad_sequences(new_steps_seq, maxlen=steps_maxlen)

In [75]:
# Mappa per ingredienti e passi
def get_feature_names(tokenizer, maxlen):
    index_word = {index: word for word, index in tokenizer.word_index.items()}
    return [index_word.get(i, f'unknown_{i}') for i in range(1, maxlen + 1)]

new_ingredient_names = get_feature_names(tokenizer_ingredients, ingredient_maxlen)
new_steps_names = get_feature_names(tokenizer_steps, steps_maxlen)

# Creazione di DataFrame
new_ingredients_df = pd.DataFrame(new_ingredients_padded, columns=new_ingredient_names)
new_steps_df = pd.DataFrame(new_steps_padded, columns=new_steps_names)

# Combinazione delle caratteristiche e dei rating
new_features_df = pd.concat([new_ingredients_df, new_steps_df], axis=1)

In [ ]:
# Calcolo della stima dei rating basata sulle correlazioni
rating_estimates = new_features_df.dot(loaded_correlation)
rating_estimates = rating_estimates / loaded_correlation.abs().sum()

print(rating_estimates)
print(type(rating_estimates))


In [ ]:
# Trova l'indice del valore massimo
max_index = rating_estimates.idxmax()

print(f"L'indice del valore massimo è: {max_index}")

In [ ]:
#ricetta con valore massimo usando la correlation matrix
print(generated_recipes[max_index])